# Filtering an audio file

In this example we are going to take a look at how to
filter away background noise in a sound file.

### Parameters
First we set up
the parameters:

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile    # For opening the sound file
from scipy import signal      # For the signal processing
import IPython                  # For playing the sound file
# plt.rcParams['figure.figsize'] = [10, 5] # Larger figures


### Listing to the signal

In [ ]:
nasa = 'apollo11_launch.wav'
IPython.display.Audio(nasa)
    

### Goal

The goal is to reduce the background noise

### Reading the signal

In [ ]:
rate, data = wavfile.read(nasa) # rate is the sample rate used in the sound file, which for this file is 44 100 Hz
data = data[:,0] # We are only using one channel

t = np.arange(0,len(data)/rate,1/rate)
plt.figure()
plt.plot(t,data,linewidth=0.2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.tight_layout()

### Taking the FFT

In [ ]:
Nfft = 2**14  # Number of points in FFT
X_1 = (1/(2*rate)) * np.fft.fftshift(np.fft.fft(data, n=Nfft))        # FFT
f = np.arange(-Nfft/2, (Nfft/2))*rate/Nfft            # frequency scale
m = np.abs(X_1)

plt.figure()
plt.plot(f, 20*m,'-')
plt.xlim(0,2000)
#plt.axis((-500, 500, 0, 11))
plt.grid()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')
plt.title('Complex magnitude spectrum')
plt.show()

### Finding the frequencies 
We will take STFT of the data to be able to plot
the spectrogram

In [ ]:
N=1024

F, T, Sxx = signal.stft(data, fs=rate, nperseg=N,noverlap=N/2,window=signal.get_window('hann',N), padded=False, boundary=None)

We now plot the spectrogram on dB scale

In [ ]:
Sxx[Sxx==0]=1e-14    # Replace zero with something small as we are going to take the logarithm
M=20 * np.log10( np.abs(Sxx))

plt.figure()
plt.pcolormesh(T,F,M,vmin=-10,shading='auto') # Plot with minimum set at -10 dB
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0,6000) # The wav only contains frequencies up to 6 000 Hz

cbar=plt.colorbar()
cbar.set_label('Magnitude (dB)')
plt.tight_layout()
plt.show()

## Filter the signal 

We now design a FIR high pass filter based on a hanning
window, dampening of 60 dB and pass above 400 Hz

In [ ]:
taps = signal.firwin(301, cutoff = 400, window = "hann",fs=rate, pass_zero=False)
nyq_rate=rate/2

plt.figure()

# Plot the frequency response of the filter
w, h = signal.freqz(taps, worN=2048)
plt.plot((w/np.pi)*nyq_rate, 20*np.log10(np.abs(h)), linewidth=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain (dB)')
plt.title('Frequency Response')
plt.xlim(0,1000)
plt.grid(True)


We now apply this filter to the audio with some added zeros at the start to
start the filter off

In [ ]:
NN=2000
dataZ=np.zeros(len(data)+NN)
dataZ[NN-1:-1]=data
filtered = signal.convolve(dataZ,taps,mode='valid')
# Remove the zeros
filtered = filtered[NN-len(taps):-1]
plt.figure()
plt.plot(t,filtered,linewidth=0.2)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.tight_layout()
plt.show()

The resulting filtered audio

In [ ]:
IPython.display.Audio(filtered,rate=rate)


The original

In [ ]:
IPython.display.Audio(nasa)


In [ ]:
N=1024

FF, TF, SxxF = signal.stft(filtered, fs=rate, nperseg=N,noverlap=N/2,window=signal.get_window('hann',N), padded=False, boundary=None)
SxxF[SxxF==0]=1e-14    # Replace zero with something small as we are going to take the logarithm
MF=20 * np.log10( np.abs(SxxF))

plt.figure()
plt.pcolormesh(TF,FF,MF,vmin=-10,shading='auto') # Plot with minimum set at -10 dB
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0,6000) # The wav only contains frequencies up to 6 000 Hz

cbar=plt.colorbar()
cbar.set_label('Magnitude (dB)')
plt.tight_layout()
plt.show()